# enrichissement aden #1

In [2]:
import pandas as pd
from geopy.distance import vincenty

## Lecture du fichier des attaques et des points de la côte

In [3]:
shorlineAden = pd.read_csv('../data/referentiels/shorlineAden.csv',sep=';')
df = pd.read_csv('../data/donneesBrutesPirateriesLatLon.csv',sep=';')

On se retreint à la zone près du golfe d'Aden.

In [4]:
dfAden=df[(df.latitude<31.) & (df.latitude>-10.) & (df.longitude>31.) & (df.longitude<78.)]

In [5]:
shorlineAden=shorlineAden.replace({',': '.'}, regex=True)
shorlineAden['latitude']=shorlineAden['latitude'].convert_objects(convert_numeric=True)
shorlineAden['longitude']=shorlineAden['longitude'].convert_objects(convert_numeric=True)

/Users/romainphilippe/app/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:2: FutureWarning: convert_objects is deprecated.  Use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
  from ipykernel import kernelapp as app
/Users/romainphilippe/app/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:3: FutureWarning: convert_objects is deprecated.  Use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
  app.launch_new_instance()


## Function pour calculer la distance à la côte

In [10]:
lon=61.616667
lat=10.783333
threshold=10
def get_distance(lat, lon):
    tab=[]
    pointAttack=(lat, lon)
    shorlineAdenCurrent=shorlineAden[(shorlineAden.latitude<lat+threshold) & (shorlineAden.latitude>lat-threshold) & (shorlineAden.longitude>lon-threshold) & (shorlineAden.longitude<lon+threshold)]
    length=len(shorlineAdenCurrent)
    for index in range(0,length):
        pointCoast= (shorlineAdenCurrent.iloc[index]['latitude'], shorlineAdenCurrent.iloc[index]['longitude'])
        distance =vincenty(pointCoast, pointAttack).kilometers
        tab.append(distance)
    return min(tab)

In [11]:
dfAden['distance_shorline']=dfAden.apply(lambda row: get_distance(row['latitude'], row['longitude']), axis=1)

/Users/romainphilippe/app/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [12]:
dfAden.to_csv("../data/aden.csv",sep=';', index=False)